In [22]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [23]:
# Load authentication data in .env file
load_dotenv()

True

In [24]:
def get_categories():
    response = requests.get('https://developers.zomato.com/api/v2.1/categories', headers=HEADERS)
    if response.status_code not in (200, 400):
        raise Exception('Failed to get the data')
    else:
        return response.status_code, response.json()
    
def read_categories(json_data):
    data = {idx:row['categories'] for idx, row in enumerate(json_data['categories'])}
    return pd.DataFrame.from_dict(data, orient='index')

In [25]:
def get_cities(list_cities):
    params = {'q' : ','.join(list_cities)}
    response = requests.get('https://developers.zomato.com/api/v2.1/cities', headers=HEADERS, params=params)
    if response.status_code != 200:
        raise Exception('Failed to get the data')
    else:
        return response.status_code, response.json()

def read_cities(json_data):
    data = {idx:row for idx, row in enumerate(json_data['location_suggestions'])}
    return pd.DataFrame.from_dict(data, orient='index')

In [106]:
def get_collections(list_city_id):
    all_data = []
    for record in list_city_id:
        params = {'city_id' : record, 'count' : 1000}
        response = requests.get('https://developers.zomato.com/api/v2.1/collections', headers=HEADERS, params=params)
        if response.status_code != 200:
            raise Exception('Failed to get the data')
        else:
            all_data.append(response.json())
    return all_data

def read_collections(json_data, list_city_id):
    data = dict()
    row = 0
    for i in range(len(list_city_id)):
        record = dict()
        for j in range(len(json_data[i]['collections'])):
            record.update({'city_id' : list_city_id[i]})
            record.update(json_data[i]['collections'][j]['collection'])
            data.update({row : record})
            row += 1
    return pd.DataFrame.from_dict(data, orient='index')

In [110]:
def get_search(list_cities):
    params = 
    response = requests.get('https://developers.zomato.com/api/v2.1/search', header=HEADERS, params=params)

def read_search():
    pass

In [108]:
def connect_db():
    engine = create_engine(f"postgresql+psycopg2://{os.getenv('db-user')}:{os.getenv('db-pwd')}@{os.getenv('db-host')}")
    conn = engine.connect()
    return engine, conn

def close_db(engine, conn):
    conn.close()
    engine.dispose()
    
def save_to_db(data, table_name):
    engine, conn = connect_db()
    data.to_sql(table_name, conn, if_exists='replace', index=False)
    close_db(engine, conn)

In [109]:
HEADERS = headers = {'user-key' : os.getenv('user-key')}
list_cities = ['Jakarta', 'Bandung', 'Bali']

# df_categories = read_categories(get_categories()[1])
# df_cities = read_cities(get_cities(list_cities)[1])
df_collections = read_collections(get_collections(df_cities['id']), df_cities['id'])

save_to_db(df_collections, 'collections')

In [35]:
import requests

def city(): # dibuat menjadi fungsi karena akan dipakai 2x agar lebih efisien
    global entityId, citySuggName # dibuat menjadi global karena diperlukan di proses selanjutnya

    cityName, entityId = input("Masukkan nama kota : "), int()
    while not entityId: # entityId dijadikan determinan loop, dimana ia kemudian akan di ubah di proses selanjutnya apabila kota inputan user terdaftar
        reqLoc = requests.get(host + f"locations?query={cityName}", headers = headers).json()
        if reqLoc["location_suggestions"] == []:
            cityName = input("Nama kota yang Anda masukkan tidak terdaftar. Silahkan ulangi memberi masukkan : ")
        else:
            entityId = reqLoc["location_suggestions"][0]["entity_id"] # mengubah entityId sehingga kondisi while loop tidak terpenuhi lagi (exit loop)
            citySuggName = reqLoc["location_suggestions"][0]["title"]

def queryCount(query): # dibuat menjadi fungsi karena akan dipakai 2x agar lebih efisien
    global qC

    qC = input(f"Masukkan jumlah {query} yang akan ditampilkan : ")
    while qC or not qC:
        try:
            qC = int(qC) # input user harus bisa dijadikan int (hanya digit bulat)
            break
        except:
            qC = input(f"Anda salah memberi masukkan. Masukkan kembali jumlah {query} yang Anda ingin tampilkan : ")

            
            
print('''\n                                  # # # # # # # 
                                #               #
                               #                 #
                              #                   #
                              #  T O M A T O E S  #
                              #                   #
                               #                 #
                                #               #
                                 #             #
                                  #           #
                                    # # # # # ''') 
print("\n                                  W E L C O M E ")

ops = input("Silahkan pilih opsi di bawah ini: \n 1. Mencari Resto \n 2. Menu Harian \n Masukkan : ")
while (ops != "1") and (ops != "2"): # memaksa user memasukkan input 1 atau 2 saja
    ops = input("Anda salah memberi masukkan. Silahkan ulangi dengan memasukkan angka 1 untuk opsi 'Mencari Resto' atau angka 2 untuk opsi 'Menu Harian' \n Masukkan : ")

host, headers = "https://developers.zomato.com/api/v2.1/", {"user-key" : "3ec56806c64b2bf086316bf2cf94ff51"}
if ops == "1": # jika user pilih kondisi 1
    city() # untuk mendapatkan entityId, citySuggName
    print(f"'{citySuggName}' dipilih.")

    queryCount("restoran") # untuk mendapatkan qC

    sort = input("Apakah Anda ingin mengurutkan hasil berdasarkan rating tertinggi? \n Y / T \n Masukkan : ").upper()
    while (sort != "Y") and (sort != "T"): # memaksa user untuk hanya menginput Y atau T
        sort = input("Anda salah memberi masukkan. Ulangi kembali masukkan dengen mengetik Y untuk 'Ya' atau T untuk 'Tidak' \n Masukkan : ")

    sortDec = "" if sort == "T" else "&sort=rating&order=desc" # jika user pilih Y maka sort dan order dimasukkan, vice versa
    req = requests.get(host + f"search?entity_id={entityId}&entity_type=city" + sortDec, headers = headers).json()
    
    if len(req["restaurants"]) != 0:
        if len(req["restaurants"]) < qC:
            print(f"Maaf, namun sistem hanya mampu menampilkan {len(req['restaurants'])} restoran. Berikut daftarnya : \n")
        else:
            print("Berikut daftar restoran tersedia : \n")
        
        try: # try dan except digunakan untuk memenuhi asumsi bila qC (jumlah inputan user) melebihi jumlah yang tersedia hasil dari API
            for resto in range(qC):
                currentResto = req["restaurants"][resto]["restaurant"]
                print(f'''Nama restoran : {currentResto["name"]}
Tipe restoran : {", ".join(currentResto["establishment"])}
Tipe makanan dan minuman : {currentResto["cuisines"]}
Alamat : {currentResto["location"]["address"]}
No. Telepon : {currentResto["phone_numbers"].replace(" ", "")}
Rating : {currentResto["user_rating"]["aggregate_rating"]}
Review : {currentResto["all_reviews_count"]:,}\n''')
        except: # menghindari error dengan tidak melakukan apa - apa
            pass
    else:
        print("Maaf, sistem tidak menemukan daftar restoran di kota ini.")

else:
    city()
    print(f"'{citySuggName}' dipilih.")

    req = requests.get(host + f"search?entity_id={entityId}&entity_type=city", headers = headers).json()
    restoName, det = input("Masukkan nama restoran : ").lower(), 1
    while det: # det adalah variabel determinan untuk menjaga keberlangsungan while loop. det di define 1 pada awalnya untuk memulai loop
        for resto in req["restaurants"]:
            if restoName in resto["restaurant"]["name"].lower(): # jika nama restoran ada, maka det diubah menjadi 0 (False)
                restoId, det = resto["restaurant"]["R"]["res_id"], 0
                break # keluar loop for
        if det: # jika det masih sama dengan 1, maka memaksa user menginput kembali nama kotanya     
            restoName = input(f"Restoran yang Anda cari tidak terdaftar di {citySuggName}. Silahkan masukkan nama restoran yang lain : ")
        else: # jika det = 0 maka keluar while loop
            break
    
    reqDailyMenu = requests.get(host + f"dailymenu?res_id={restoId}", headers = headers)
    if reqDailyMenu.status_code == 200: # status code 200 tidak menjamin ada daily menu. maka dari itu, disisipkan kondisi lagi di dalamnya
        menuCount = reqDailyMenu.json()["daily_menus"][0]["daily_menu"]["dishes"]

        if menuCount != []: # jika len(menuCount) > 0
            queryCount("menu")
            if len(menuCount) < qC:
                print(f"Maaf, namun sistem hanya mampu menampilkan {len(menuCount)} menu harian. Berikut daftarnya : \n")
            else:
                print("Berikut daftar menu harian tersedia : \n")
            
            try:
                for x in range(qC):
                    print("{}".format(',\n'.join([menuCount[x]['dish']['name']])))
            except:
                pass
        else: # jika len(menuCount) == 0
            print("Maaf, sistem tidak menemukan daftar menu harian di restoran ini.")
    elif (reqDailyMenu.status_code == 400): # tidak ditemukan daftar menu di API
        print("Maaf, sistem tidak menemukan daftar menu harian di restoran ini.")
    else: # jika status code diluar 200 atau 400, misal 502 Bad Gateway, maka diasumsikan terjadi kesalahan jaringan
        print("Terjadi kesalahan pada jaringan. Silahkan ulangi kembali dalam beberapa waktu atau hubungi penyedia layanan.")



                                  # # # # # # # 
                                #               #
                               #                 #
                              #                   #
                              #  T O M A T O E S  #
                              #                   #
                               #                 #
                                #               #
                                 #             #
                                  #           #
                                    # # # # # 

                                  W E L C O M E 
Silahkan pilih opsi di bawah ini: 
 1. Mencari Resto 
 2. Menu Harian 
 Masukkan : 1
Masukkan nama kota : Bali
'Bali' dipilih.
Masukkan jumlah restoran yang akan ditampilkan : 10
Apakah Anda ingin mengurutkan hasil berdasarkan rating tertinggi? 
 Y / T 
 Masukkan : Y
Berikut daftar restoran tersedia : 

Nama restoran : Pork Star
Tipe restoran : Quick Bites
Tipe makanan dan minuman : Asian, Indon

In [36]:
test = int()
test

0